In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from matplotlib.pyplot import figure

figure(figsize=(16, 12), dpi=80)

root = 'webcam'

#crops the images edges randomly
def directionCrop(img):
    toReturn = [img]
    height = img.shape[0]
    width = img.shape[1]
    height10 = int(height * .1)
    width10 = int(width * .1)
    heightStarts = [0, height10]
    heightEnds = [height - height10, height]
    widthStarts = [0, width10]
    widthEnds = [width - width10, width]
    for hStart in heightStarts:
        for hEnd in heightEnds:
            for wStarts in widthStarts:
                for wEnds in widthEnds:
                    roll = random.random()
                    if roll > .6:#40% chance
                        toReturn.append(img[hStart:hEnd, wStarts: wEnds])
    return toReturn

#changes the angle of the images, filling the new black space with noise
def angleChange(img, a):
    whiteImg = np.ones(img.shape)
    height, width = img.shape[:2]
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=1)
    rotated_image = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))
    whiteImg =  cv2.warpAffine(src=whiteImg, M=rotate_matrix, dsize=(width, height))
    whiteImg = np.where(whiteImg > 0 , 0, 1)
    whiteImg = whiteImg *  np.random.uniform(0,1, img.shape)
    whiteImg *= 255
    whiteImg = whiteImg.astype('uint8')
    rotated_image += whiteImg
    return rotated_image

#builds the augmented versions of an image
def preProcess(img):
    toReturn = []
    imgs = directionCrop(img)
    angles = [-3, 0, 3]
    for originalImage in imgs:
        w = 160
        h = 160
        originalImage = cv2.resize(originalImage, (w,h), interpolation = cv2.INTER_AREA)
        for a in angles:
            if a != 0:
                i = angleChange(originalImage, a)
            else:
                i = originalImage
    #         plt.imshow(resized)
    #         plt.show()
    #         z = 1/0
            toReturn.append(i)
            mean = 0
            var = 30
            sigma = var**0.5
            gauss = np.random.normal(mean,sigma,(h,w,3))
    #         gauss = gauss.reshape(h,w, 3)
            noisy = i + gauss
            toReturn.append(noisy)
    return toReturn

#resets the folders
def wipeFolders():
    roots = ['webcamTrain', 'webcamTest']
    classes = ['stop', 'go']
    for r in roots:
        for c in classes:
            if os.path.isdir(f'{r}/{c}'):
                for fName in os.listdir(f'{r}/{c}'):
                    os.remove(f'{r}/{c}/{fName}')
            else:
                os.mkdir(f'{r}/{c}')

#processes a whole folder to make augmented images
def batchProcess(root):
    wipeFolders()
    trainImg = []
    trainLabel = []
    for label in os.listdir(root):
        for idx, img in enumerate(os.listdir(f'{root}/{label}')):
            roll = random.random()
            outputFolder = 'webcamTrain' if roll > .2 else 'webcamTest'#.8/.2 train test split
            img = cv2.imread(f'{root}/{label}/{img}', cv2.IMREAD_COLOR)
            cropped = preProcess(img)
            for idy, c in enumerate(cropped):
                cv2.imwrite(f'{outputFolder}/{label}/{idx}-{idy}.png', c)        
    
batchProcess(root)

<Figure size 1280x960 with 0 Axes>